In [1]:
import netCDF4 as nc
import json
import numpy as np
import pandas as pd
import plotly.express as px

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Fix winding of json file and import as geojson
import json
import geojson_rewind
import geopandas as gpd

sa4_gdf = gpd.read_file("data/SA4_2021_AUST_GDA2020.json")

sa4_gdf = sa4_gdf.set_geometry(
    gpd.GeoDataFrame.from_features(
        json.loads(
            geojson_rewind.rewind(
                sa4_gdf.to_json(), 
                rfc7946=False
            )
        )["features"]
    ).geometry
)

location_codes = ["301", "302", "303", "304", "305", "309", "310", "311", "313", "314", "316", "317"]
sa4_gdf = sa4_gdf[sa4_gdf["SA4_CODE21"].isin(location_codes)]

print(sa4_gdf.shape)
sa4_gdf.head()

(12, 13)


,SA4_CODE21,SA4_NAME21,CHG_FLAG21,CHG_LBL21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,301,Brisbane - East,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,653.1329,http://linked.data.gov.au/dataset/asgsed3/SA4/301,"MULTIPOLYGON (((153.22243 -27.39012, 153.22263..."
1,302,Brisbane - North,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,186.9483,http://linked.data.gov.au/dataset/asgsed3/SA4/302,"POLYGON ((153.00136 -27.41543, 153.00134 -27.4..."
2,303,Brisbane - South,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,265.3445,http://linked.data.gov.au/dataset/asgsed3/SA4/303,"POLYGON ((153.17577 -27.60669, 153.17485 -27.6..."
3,304,Brisbane - West,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,269.6508,http://linked.data.gov.au/dataset/asgsed3/SA4/304,"MULTIPOLYGON (((152.96542 -27.54462, 152.96547..."
4,305,Brisbane Inner City,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,81.7393,http://linked.data.gov.au/dataset/asgsed3/SA4/305,"MULTIPOLYGON (((153.06952 -27.44148, 153.06881..."


In [3]:
years = ["2020-2039", "2040-2059", "2060-2079", "2080-2099"]
historical_dataset = nc.Dataset('data/hd.climatology.ccam10bc_ACCESS1-0Q_rcp85.1986-2005.nc')

lons = historical_dataset["lon"][:]
lats = historical_dataset["lat"][:]
combination_points = np.array(np.meshgrid(lons, lats)).T.reshape(-1, 2)

data = historical_dataset["hd_annual"][0,:,:].flatten(order="F")

temperature_gdf = gpd.GeoDataFrame(
    data,
    columns=["historical (1986-2005)"],
    geometry=gpd.points_from_xy(combination_points[:,0], combination_points[:,1])
)

for year in years:
    heat_dataset = nc.Dataset('data/hd.absolute-change.ccam10bc_ACCESS1-0Q_rcp85.{}_minus_1986-2005.nc'.format(year))
    data = heat_dataset["hd_annual"][0,:,:].flatten(order="F")
    # Add change to historical to give new number of days above 35c
    data = np.add(data, temperature_gdf["historical (1986-2005)"])
    temperature_gdf[year] = data

print(temperature_gdf.shape)
temperature_gdf.head()

/Users/s5068337/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


(48841, 6)


,historical (1986-2005),geometry,2020-2039,2040-2059,2060-2079,2080-2099
0,365.0,POINT (132.00000 -32.00000),365.0,365.0,365.0,365.0
1,365.0,POINT (132.00000 -31.90000),365.0,365.0,365.0,365.0
2,365.0,POINT (132.00000 -31.80000),365.0,365.0,365.0,365.0
3,365.0,POINT (132.00000 -31.70000),365.0,365.0,365.0,365.0
4,365.0,POINT (132.00000 -31.60000),365.0,365.0,365.0,365.0


In [4]:
# Find which points lie within each sa4 area
points_within = gpd.sjoin(temperature_gdf, sa4_gdf, how='inner')
points_within.head()

/var/folders/8q/byh6pjkx74d7p_crfrzgdpbd2xgt0t/T/ipykernel_2447/3657317686.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  points_within = gpd.sjoin(temperature_gdf, sa4_gdf, how='inner')


,historical (1986-2005),geometry,2020-2039,2040-2059,2060-2079,2080-2099,index_right,SA4_CODE21,SA4_NAME21,CHG_FLAG21,CHG_LBL21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21
43801,9.80,POINT (151.80000 -27.70000),13.6,27.349998,33.500000,47.450001,16,317,Toowoomba,0,No change,3RQLD,Rest of Qld,3,Queensland,AUS,Australia,2258.7953,http://linked.data.gov.au/dataset/asgsed3/SA4/317
43802,9.90,POINT (151.80000 -27.60000),14.9,27.699999,33.599998,47.599998,16,317,Toowoomba,0,No change,3RQLD,Rest of Qld,3,Queensland,AUS,Australia,2258.7953,http://linked.data.gov.au/dataset/asgsed3/SA4/317
44022,8.10,POINT (151.90000 -27.70000),11.6,22.549999,28.200001,40.800003,16,317,Toowoomba,0,No change,3RQLD,Rest of Qld,3,Queensland,AUS,Australia,2258.7953,http://linked.data.gov.au/dataset/asgsed3/SA4/317
44023,4.35,POINT (151.90000 -27.60000),6.9,12.799999,17.500000,24.650000,16,317,Toowoomba,0,No change,3RQLD,Rest of Qld,3,Queensland,AUS,Australia,2258.7953,http://linked.data.gov.au/dataset/asgsed3/SA4/317
44024,7.75,POINT (151.90000 -27.50000),12.3,23.650000,28.100000,42.250000,16,317,Toowoomba,0,No change,3RQLD,Rest of Qld,3,Queensland,AUS,Australia,2258.7953,http://linked.data.gov.au/dataset/asgsed3/SA4/317


In [5]:
# For each region, calculate average hd_annaul
def find_average_hd_annual(area_code, year):
    points = points_within[points_within["SA4_CODE21"] == area_code]
    return points[year].mean()

years = ["historical (1986-2005)", "2020-2039", "2040-2059", "2060-2079", "2080-2099"]
for year in years:
    sa4_gdf[year] = sa4_gdf["SA4_CODE21"].apply(lambda area_code: find_average_hd_annual(area_code, year))

sa4_gdf.set_index("SA4_CODE21", drop=False, inplace=True)
sa4_gdf.head()

,SA4_CODE21,SA4_NAME21,CHG_FLAG21,CHG_LBL21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,historical (1986-2005),2020-2039,2040-2059,2060-2079,2080-2099
SA4_CODE21,,,,,,,,,,,,,,,,,,
301,301,Brisbane - East,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,653.1329,http://linked.data.gov.au/dataset/asgsed3/SA4/301,"MULTIPOLYGON (((153.22243 -27.39012, 153.22263...",0.810000,1.700000,9.150000,22.260000,63.790001
302,302,Brisbane - North,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,186.9483,http://linked.data.gov.au/dataset/asgsed3/SA4/302,"POLYGON ((153.00136 -27.41543, 153.00134 -27.4...",1.400000,2.075000,4.475000,7.925000,25.575001
303,303,Brisbane - South,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,265.3445,http://linked.data.gov.au/dataset/asgsed3/SA4/303,"POLYGON ((153.17577 -27.60669, 153.17485 -27.6...",1.983333,4.016666,8.916667,14.450001,30.716667
304,304,Brisbane - West,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,269.6508,http://linked.data.gov.au/dataset/asgsed3/SA4/304,"MULTIPOLYGON (((152.96542 -27.54462, 152.96547...",3.100000,6.325000,13.487499,22.700001,47.662498
305,305,Brisbane Inner City,0,No change,3GBRI,Greater Brisbane,3,Queensland,AUS,Australia,81.7393,http://linked.data.gov.au/dataset/asgsed3/SA4/305,"MULTIPOLYGON (((153.06952 -27.44148, 153.06881...",NaN,NaN,NaN,NaN,NaN


In [6]:
# Fix cases where there is no data points in region
# Take the closest data point to the centoid of region
from shapely.ops import nearest_points

def nearestPoint(point):
    # find the nearest point and return row
    points = temperature_gdf.geometry.unary_union
    nearest = temperature_gdf.geometry == nearest_points(point, points)[1]
    return temperature_gdf[nearest]

def fixMissingData(row):
    if not np.isnan(row["historical (1986-2005)"]):
        return row
    centroid = row["geometry"].centroid
    nearest = nearestPoint(centroid)
    for year in years:
        row[year] = nearest[year].values[0]
    return row
    
sa4_gdf = sa4_gdf.apply(fixMissingData, axis=1)

In [7]:
# Convert wide DF to tall one for visualization
sa4_gdf_tall = pd.melt(sa4_gdf, id_vars=['SA4_NAME21', "SA4_CODE21"], value_vars=years)
print(sa4_gdf_tall.shape)
sa4_gdf_tall.head()

(60, 4)


,SA4_NAME21,SA4_CODE21,variable,value
0,Brisbane - East,301,historical (1986-2005),0.810000
1,Brisbane - North,302,historical (1986-2005),1.400000
2,Brisbane - South,303,historical (1986-2005),1.983333
3,Brisbane - West,304,historical (1986-2005),3.100000
4,Brisbane Inner City,305,historical (1986-2005),2.050000


In [ ]:
fig = px.choropleth_mapbox(sa4_gdf_tall,
                           geojson=sa4_gdf.geometry,
                           locations=sa4_gdf_tall.SA4_CODE21,
                           hover_name=sa4_gdf_tall.SA4_NAME21,
                           hover_data={
                               'value': True,
                               'SA4_CODE21': False,
                               'variable': False
                           },
                           color="value",
                           color_continuous_scale=["green", "yellow", "#edc40c", "#de3a2f", "#960202"],
                           labels={"value" : "Annual number of hot days (>35c)"},
                           animation_frame="variable",
                           range_color=(0, 60),
                           mapbox_style="carto-positron",
                           zoom=7,
                           center = {"lat": -27.2500, "lon": 153.0260},
                           opacity=0.5,
                           height=700
                          )

# Set annimation speed (in ms)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000

# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()